# Efficient Warehousing - Inventory Cost Management and Market Basket Analysis

Problem Statement:

In recent times, the e-commerce sector has experienced significant growth and success, offering customers the convenience of a wide array of product choices that can be conveniently ordered from the comfort of their homes. However, the question arises as to how these e-commerce companies are able to provide such an extensive range of products. To meet customer demands, these companies must maintain substantial inventories in their warehouses. Within these inventories, some products are classified as fast-moving, with high sales velocity, while others are slow-moving. The storage and maintenance of each product entail costs for the company in terms of space and upkeep. Consequently, it becomes imperative for organizations to effectively plan their inventory to mitigate unnecessary expenses.

OList, an e-commerce enterprise, has recently faced financial losses and aims to optimize its inventory management to minimize avoidable costs. The objective of this assignment is to oversee the inventory cost management of OList. The key tasks involve identifying the top revenue-contributing products and utilizing market basket analysis to examine customer purchasing behavior. By doing so, it becomes possible to estimate, with a reasonable degree of certainty, which items are more likely to be purchased individually or in conjunction with other products.

# Data Exploration and Cleaning

In [1]:
# importing libraries

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#Reading orders sheet data
orders = pd.read_excel("Retail_dataset.xlsx", sheet_name = "orders")

In [3]:
orders.shape

(99441, 7)

In [4]:
orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_timestamp,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,7c396fd4830fd04220f754e42b4e5bff,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-10 21:25:13,2017-10-18
1,53cdb2fc8bc7dce0b6741e2150273451,af07308b275d755c9edb36a90c618231,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-08-07 15:27:45,2018-08-13
2,47770eb9100c2d0c44946d9cf07ec65d,3a653a41f6f9fc3d2a113cf8398680e8,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-17 18:06:29,2018-09-04
3,949d5b44dbf5de918fe9c16f97b45f8a,7c142cf63193a1473d2e66489a9ae977,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-12-02 00:28:42,2017-12-15
4,ad21c59c0840e6cb83a9ceb5573f8159,72632f0f9dd73dfee390c9b22eb56dd6,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-16 18:17:02,2018-02-26


For this project, we are only considering the order with status as 'delivered'.

In [5]:
orders = orders[orders.order_status == "delivered"]

In [13]:
#Checking null values
orders.isnull().sum()

order_id                         0
customer_id                      0
order_status                     0
order_purchase_timestamp         0
order_approved_at                0
order_delivered_timestamp        0
order_estimated_delivery_date    0
dtype: int64

The dataset contains two columns, namely "order_approved_at" and "order_delivered_timestamp" with missing values. To address this, we will perform imputation techniques.

For the "order_approved_at" column, we will assume that the missing values correspond to the same time as the "order_purchase_timestamp".

Regarding the "order_delivered_timestamp" column, we will assume that the missing values align with the "order_estimated_delivery_date."

By applying these imputations, we aim to fill in the null values in these columns and ensure the completeness of the dataset.

In [7]:
#Imputing the null values of the two columns "order_approved_at" and "order_delivered_timestamp"
orders.order_approved_at.fillna(orders.order_purchase_timestamp, inplace = True)
orders.order_delivered_timestamp.fillna(orders.order_estimated_delivery_date, inplace = True)

In [8]:
#Reading the order items data
order_items = pd.read_excel("Retail_dataset.xlsx", sheet_name = "order_items")

In [9]:
order_items.shape

(112650, 6)

In [10]:
order_items.head()

,order_id,order_item_id,product_id,seller_id,price,shipping_charges
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,199.90,18.14


In [12]:
#Checking for null values 
order_items.isnull().sum()

order_id            0
order_item_id       0
product_id          0
seller_id           0
price               0
shipping_charges    0
dtype: int64

In [14]:
#Reading the customers data
customers = pd.read_excel("Retail_dataset.xlsx", sheet_name = "customers")

In [39]:
customers.shape

(99441, 4)

In [40]:
customers.head()

,customer_id,customer_zip_code_prefix,customer_city,customer_state
0,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [41]:
#Checking null values
customers.isna().sum()

customer_id                 0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64

In [42]:
#Checking for duplicate customer_id
customers.customer_id.duplicated().sum()

3345

In [43]:
#Dropping duplicate customer_id and keeping only the first one
customers.drop_duplicates(subset="customer_id", keep="first", inplace = True)

In [44]:
#Reading payments data
payments = pd.read_excel("Retail_dataset.xlsx", sheet_name = "payments")

In [45]:
payments.shape

(103886, 5)

In [46]:
payments.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [47]:
#Checking null values
payments.isna().sum().sort_values(ascending = False)

order_id                0
payment_sequential      0
payment_type            0
payment_installments    0
payment_value           0
dtype: int64

In [48]:
#Reading products data
products = pd.read_excel("Retail_dataset.xlsx", sheet_name = "products")

In [49]:
products.shape

(32951, 6)

In [50]:
products.head()

,product_id,product_category_name,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumery,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,art,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,sports_leisure,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,baby,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,housewares,625.0,20.0,17.0,13.0


In [51]:
#Checking null values
products.isnull().sum()

product_id                 0
product_category_name    170
product_weight_g           2
product_length_cm          2
product_height_cm          2
product_width_cm           2
dtype: int64

In [52]:
products

,product_id,product_category_name,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumery,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,art,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,sports_leisure,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,baby,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,housewares,625.0,20.0,17.0,13.0
...,...,...,...,...,...,...
32946,a0b7d5a992ccda646f2d34e418fff5a0,toys,12300.0,40.0,40.0,40.0
32947,bf4538d88321d0fd4412a93c974510e6,toys,1700.0,16.0,19.0,16.0
32948,9a7c6041fa9592d9d9ef6cfe62a71f8c,toys,1400.0,27.0,7.0,27.0
32949,83808703fc0706a22e264b9d75f04a2e,toys,700.0,31.0,13.0,20.0


In [53]:
#Removing rows with null product category name
products = products[~products.product_category_name.isnull()]

In [55]:
products.isnull().sum()

product_id               0
product_category_name    0
product_weight_g         2
product_length_cm        2
product_height_cm        2
product_width_cm         2
dtype: int64

In [57]:
products[products.product_weight_g.isnull()]

,product_id,product_category_name,product_weight_g,product_length_cm,product_height_cm,product_width_cm
8578,09ff539a621711667c43eba6a3bd8466,toys,NaN,NaN,NaN,NaN
18851,5eb564652db742ff8f28759cd8d2652a,toys,NaN,NaN,NaN,NaN


Since null values of column is of toys category, will replace null values with median of toys category.

In [68]:
#Imputing the null values with the median of the column values of that category
products.product_weight_g.fillna(products[products.product_category_name == 'toys']['product_weight_g'].median(), inplace = True)
products.product_length_cm.fillna(products[products.product_category_name == 'toys']['product_length_cm'].median(), inplace = True)
products.product_height_cm.fillna(products[products.product_category_name == 'toys']['product_height_cm'].median(), inplace = True)
products.product_width_cm.fillna(products[products.product_category_name == 'toys']['product_width_cm'].median(), inplace = True)

## Merging Dataframes
orders, orders_items,products,customer,payments

In [95]:
ord_prod =pd.merge(order_items, products, on="product_id", how="inner", indicator=True)
ord_prod.head()

,order_id,order_item_id,product_id,seller_id,price,shipping_charges,product_category_name,product_weight_g,product_length_cm,product_height_cm,product_width_cm,_merge
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,58.9,13.29,toys,650.0,28.0,9.0,14.0,both
1,130898c0987d1801452a8ed92a670612,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,55.9,17.96,toys,650.0,28.0,9.0,14.0,both
2,532ed5e14e24ae1f0d735b91524b98b9,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,64.9,18.33,toys,650.0,28.0,9.0,14.0,both
3,6f8c31653edb8c83e1a739408b5ff750,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,58.9,16.17,toys,650.0,28.0,9.0,14.0,both
4,7d19f4ef4d04461989632411b7e588b9,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,58.9,13.29,toys,650.0,28.0,9.0,14.0,both


In [96]:
cust_ord =pd.merge(orders, customers, on="customer_id", how="inner", indicator=True)
cust_ord.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_timestamp,order_estimated_delivery_date,customer_zip_code_prefix,customer_city,customer_state,_merge
0,e481f51cbdc54678b7cc49136f2d6af7,7c396fd4830fd04220f754e42b4e5bff,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-10 21:25:13,2017-10-18,3149,sao paulo,SP,both
1,69923a4e07ce446644394df37a710286,7c396fd4830fd04220f754e42b4e5bff,delivered,2017-09-04 11:26:38,2017-09-04 11:43:25,2017-09-05 19:20:20,2017-09-15,3149,sao paulo,SP,both
2,53cdb2fc8bc7dce0b6741e2150273451,af07308b275d755c9edb36a90c618231,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-08-07 15:27:45,2018-08-13,47813,barreiras,BA,both
3,47770eb9100c2d0c44946d9cf07ec65d,3a653a41f6f9fc3d2a113cf8398680e8,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-17 18:06:29,2018-09-04,75265,vianopolis,GO,both
4,949d5b44dbf5de918fe9c16f97b45f8a,7c142cf63193a1473d2e66489a9ae977,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-12-02 00:28:42,2017-12-15,59296,sao goncalo do amarante,RN,both


In [97]:
ord_pay = pd.merge(cust_ord, payments, on="order_id", how="inner")
ord_pay.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_timestamp,order_estimated_delivery_date,customer_zip_code_prefix,customer_city,customer_state,_merge,payment_sequential,payment_type,payment_installments,payment_value
0,e481f51cbdc54678b7cc49136f2d6af7,7c396fd4830fd04220f754e42b4e5bff,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-10 21:25:13,2017-10-18,3149,sao paulo,SP,both,1,credit_card,1,18.12
1,e481f51cbdc54678b7cc49136f2d6af7,7c396fd4830fd04220f754e42b4e5bff,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-10 21:25:13,2017-10-18,3149,sao paulo,SP,both,3,voucher,1,2.00
2,e481f51cbdc54678b7cc49136f2d6af7,7c396fd4830fd04220f754e42b4e5bff,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-10 21:25:13,2017-10-18,3149,sao paulo,SP,both,2,voucher,1,18.59
3,69923a4e07ce446644394df37a710286,7c396fd4830fd04220f754e42b4e5bff,delivered,2017-09-04 11:26:38,2017-09-04 11:43:25,2017-09-05 19:20:20,2017-09-15,3149,sao paulo,SP,both,1,credit_card,1,44.11
4,53cdb2fc8bc7dce0b6741e2150273451,af07308b275d755c9edb36a90c618231,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-08-07 15:27:45,2018-08-13,47813,barreiras,BA,both,1,wallet,1,141.46


In [98]:
order_final = pd.merge(ord_pay, ord_prod, on="order_id", how="inner")
order_final.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_timestamp,order_estimated_delivery_date,customer_zip_code_prefix,customer_city,customer_state,...,product_id,seller_id,price,shipping_charges,product_category_name,product_weight_g,product_length_cm,product_height_cm,product_width_cm,_merge_y
0,e481f51cbdc54678b7cc49136f2d6af7,7c396fd4830fd04220f754e42b4e5bff,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-10 21:25:13,2017-10-18,3149,sao paulo,SP,...,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,29.99,8.72,housewares,500.0,19.0,8.0,13.0,both
1,e481f51cbdc54678b7cc49136f2d6af7,7c396fd4830fd04220f754e42b4e5bff,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-10 21:25:13,2017-10-18,3149,sao paulo,SP,...,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,29.99,8.72,housewares,500.0,19.0,8.0,13.0,both
2,e481f51cbdc54678b7cc49136f2d6af7,7c396fd4830fd04220f754e42b4e5bff,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-10 21:25:13,2017-10-18,3149,sao paulo,SP,...,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,29.99,8.72,housewares,500.0,19.0,8.0,13.0,both
3,69923a4e07ce446644394df37a710286,7c396fd4830fd04220f754e42b4e5bff,delivered,2017-09-04 11:26:38,2017-09-04 11:43:25,2017-09-05 19:20:20,2017-09-15,3149,sao paulo,SP,...,9abb00920aae319ef9eba674b7d2e6ff,1771297ac436903d1dd6b0e9279aa505,35.39,8.72,baby,350.0,19.0,14.0,12.0,both
4,53cdb2fc8bc7dce0b6741e2150273451,af07308b275d755c9edb36a90c618231,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-08-07 15:27:45,2018-08-13,47813,barreiras,BA,...,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,118.70,22.76,toys,400.0,19.0,13.0,19.0,both


In [99]:
order_final=order_final.drop({'_merge_x','_merge_y'},axis='columns')

In [100]:
order_final.isnull().sum()

order_id                         0
customer_id                      0
order_status                     0
order_purchase_timestamp         0
order_approved_at                0
order_delivered_timestamp        0
order_estimated_delivery_date    0
customer_zip_code_prefix         0
customer_city                    0
customer_state                   0
payment_sequential               0
payment_type                     0
payment_installments             0
payment_value                    0
order_item_id                    0
product_id                       0
seller_id                        0
price                            0
shipping_charges                 0
product_category_name            0
product_weight_g                 0
product_length_cm                0
product_height_cm                0
product_width_cm                 0
dtype: int64

# EDA

# RFM Model

RFM model, also known as Recency, Frequency, Monetary model, is a customer segmentation framework used in marketing and customer relationship management (CRM). It enables businesses to analyze and categorize customers based on their transactional behavior and value.

RFM stands for:

Recency: Refers to how recently a customer has made a purchase. It measures the time gap between a customer's last transaction and the present date. Generally, customers who have made a purchase more recently are considered more engaged and likely to make repeat purchases.

Frequency: Represents the frequency or number of purchases made by a customer within a specific period. It indicates how often a customer interacts with the business. Higher frequency typically indicates higher engagement and loyalty.

Monetary: Reflects the monetary value of a customer's purchases. It measures the total amount spent by a customer during a given period. Customers with higher monetary value contribute more revenue to the business and are often considered high-value customers.

## Recency

In [102]:
#Last date on which customer made the purchase
Recency=pd.DataFrame(order_final.groupby('product_id')['order_purchase_timestamp'].max().reset_index())

In [104]:
Recency

,product_id,order_purchase_timestamp
0,00066f42aeeb9f3007548bb9d3f33c38,2018-05-20 18:45:21
1,00088930e925c41fd95ebfe695fd2655,2017-12-12 19:20:28
2,0009406fd7479715e4bef61dd91f2462,2017-12-21 16:21:47
3,000b8f95fcb9e0096488278317764d19,2018-08-10 13:24:35
4,000d9be29b5207b54e86aa1b1ac54872,2018-04-03 09:24:12
...,...,...
32049,fff6177642830a9a94a0f2cba5e476d1,2017-09-03 11:38:54
32050,fff81cc3158d2725c0655ab9ba0f712c,2018-07-30 09:18:59
32051,fff9553ac224cec9d15d49f5a263411f,2017-10-06 16:44:05
32052,fffdb2d0ec8d6a61f0a0a0db3f25b441,2018-08-10 17:15:07
